In [1]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [3]:
# application variables
user = input('username: ')
my_password = getpass('Password: ')
search_term = input('search term: ')

# create instance of web driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()
sleep(5)
username = driver.find_element_by_xpath('//input[@name="text"]')
username.send_keys(user)
username.send_keys(Keys.RETURN)
sleep(3)

password = driver.find_element_by_xpath('//input[@name="password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(3)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text('Latest').click()

username: talhaasan18@hotmail.com
Password: ········
search term: #biden




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/talhaasan/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/38/55r5f1ds22bb6pmv70lywc3r0000gn/T/ipykernel_51851/1899189693.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/38/55r5f1ds22bb6pmv70lywc3r0000gn/T/ipykernel_51851/1899189693.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  username = driver.find_element_by_xpath('//input[@name="text"]')
/var/folders/38/55r5f1ds22bb6pmv70lywc3r0000gn/T/ipykernel_51851/1899189693.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_xpath('//input[@name="password"]')
/var/folders/38/55r5f1ds22bb6pmv70

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@aria-label="Search query"]"}
  (Session info: chrome=97.0.4692.71)
Stacktrace:
0   chromedriver                        0x0000000104677e69 chromedriver + 5160553
1   chromedriver                        0x0000000104602593 chromedriver + 4679059
2   chromedriver                        0x00000001041b62c8 chromedriver + 172744
3   chromedriver                        0x00000001041ebb62 chromedriver + 392034
4   chromedriver                        0x00000001041ebd21 chromedriver + 392481
5   chromedriver                        0x000000010421e304 chromedriver + 598788
6   chromedriver                        0x00000001042093fd chromedriver + 513021
7   chromedriver                        0x000000010421c0ab chromedriver + 589995
8   chromedriver                        0x0000000104209623 chromedriver + 513571
9   chromedriver                        0x00000001041dedce chromedriver + 3

In [4]:
# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
driver.close()

/var/folders/38/55r5f1ds22bb6pmv70lywc3r0000gn/T/ipykernel_51851/3771108767.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
/Users/talhaasan/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=97.0.4692.71)
Stacktrace:
0   chromedriver                        0x0000000104677e69 chromedriver + 5160553
1   chromedriver                        0x0000000104602593 chromedriver + 4679059
2   chromedriver                        0x00000001041b62c8 chromedriver + 172744
3   chromedriver                        0x00000001041b9141 chromedriver + 184641
4   chromedriver                        0x00000001041b8f61 chromedriver + 184161
5   chromedriver                        0x00000001041b91fc chromedriver + 184828
6   chromedriver                        0x00000001041eb876 chromedriver + 391286
7   chromedriver                        0x00000001041ebd21 chromedriver + 392481
8   chromedriver                        0x00000001041e0667 chromedriver + 345703
9   chromedriver                        0x00000001042093ad chromedriver + 512941
10  chromedriver           

In [5]:
with open('biden.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)